In [139]:
import blaze, pandas
import IPython,jinja2,json
from coffeetools import coffee
from bokeh.sampledata import iris
from IPython.display import display as display
import bokeh
import bokeh.plotting
%reload_ext yamlmagic
#bokeh.plotting.output_notebook(resources=bokeh.resources.INLINE)

In [140]:
#!bower install d3
#!bower install baobab

In [141]:
IPython.display.HTML("""<style>
#viewport {
  overflow-y: auto;
  width: 800px;
  height: 600px;
  float: left;
}
table {
    width: 750px;
}
</style>""")

## Visualize Data directly from blaze

In [142]:
def infer_type(v):
    if isinstance(v,str):
        return 'string'
    elif isinstance(v,type({})):
        return 'object'
    else:
        return 'number'

In [149]:
data = blaze.Data(iris.flowers)
#data=blaze.Data(pandas.DataFrame(pandas.np.random.randn(10000,3),columns=['x','y','z']))
df = blaze.odo(data, pandas.DataFrame)

payload = pandas.concat([df for i in range(1)],axis=0,ignore_index=True).to_dict(orient='split')
del payload['index']
payload['column_info'] = {}
for c in payload['columns']:
    payload['column_info'][c] ={ 
            'dtype': data.dshape.measure.dict[c].ty.to_numpy_dtype().name,
            'type': infer_type( payload['data'][0][payload['columns'].index(c)])
        } 

##### Configure require

In [158]:
IPython.display.Javascript(coffee.compile(jinja2.Template("""
window.require.config
    paths: 
        d3: '//d3js.org/d3.v3.min'
        jquery: '/static/components/jquery/dist/jquery.min'
        jqueryui: 'https://cdn.jsdelivr.net/jquery.ui/1.11.4/jquery-ui.min'
        baobab: 'https://cdn.rawgit.com/Yomguithereal/baobab/master/build/baobab.min'
    shim: 
        jqueryui: 
            exports: '$'
            deps: ['jquery'] 

require ['baobab'], (Baobab)->
    window.tree = new Baobab {{json.dumps(payload)}}
    tree.set 'shape', Baobab.monkey ['data'], (data)->
        [data.length,data[0].length]
    tree.set 'ColumnDataSource', {}
    tree.set 'Derived', {'data':[],'index':[]} 

    tree.get('columns').forEach (column_name)->
        tree.set ['ColumnDataSource',column_name], 
            name: column_name
            data: Baobab.monkey ['data'],['columns'],['.','name'], (data, columns, name)->
                index = columns.indexOf name
                data.map (d)-> d[index]
        methods = 'min,max,sum,mean,median,variance,deviation'.split(',')
        
    derived = tree.select ['ColumnDataSource']
    derived.set 'average', {name: 'average'}
    
    derived.set ['average','data'], Baobab.monkey  ['data'],['columns'],['column_info'], (data,cols,info)->
        value_indexes = cols.map (c,i)-> if info[c]['type'] == 'number' then i
        data.map (row)=> d3.mean value_indexes.map (i)=> row[i]
    derived.set ['deviation','data'], Baobab.monkey  ['data'],['columns'],['column_info'], (data,cols,info)->
        value_indexes = cols.map (c,i)-> if info[c]['type'] == 'number' then i
        data.map (row)=> d3.deviation value_indexes.map (i)=> row[i]

    init = 
        rows: 20
        
    tree.set 'table',
        start: 0
        rows: init['rows']
        # All available column data sources
        columns:  Baobab.monkey ['ColumnDataSource'], (cds)-> d3.keys cds
        data: Baobab.monkey ['ColumnDataSource'],['.','columns'], (cds,cols)->
            d3.zip (cols.map (d)=> cds[d].data)...
        
        sort: 
            ascending: -1
            columns: []
        index: Baobab.monkey ['.','sort','ascending'],['shape',0],['.','sort','columns'],['ColumnDataSource'], (dir,shape,cols,cds)->
            if cols.length > 0 and dir != 0
                d3.zip d3.range(0,shape),(cols.map (c)=> 
                        console.log c
                        cds[c].data)...
                    .sort (a,b)=> dir*(a[1]-b[1])
                    .map (d)-> d[0]       
            else 
                d3.range 0,shape
""").render(**globals())))

<IPython.core.display.Javascript object>

In [160]:
IPython.display.display(IPython.display.HTML("""
<div class="row">
<div><a id="undo">undo</a>|<!--a id="redo">redo</a--></div>
<div class="col col-md-12">
<div id="viewport" style="background-color:cyan;"></div>
</div>
</div>""" ),IPython.display.Javascript(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->
    window.createtable = ()->
        table = d3.select '#viewport'
            .selectAll 'table'
            .data [1]
        table.enter().append('table')
        columns = tree.get('table','columns')
        header = table.selectAll 'tr.columns'
            .data [columns]
        header.enter().append('tr').classed('columns',yes)
        header.each (d)->
            row = d3.select(@).selectAll('th').data(d)
            row.enter().append('th')
            row.each (d)->
                window.cursor = tree.select('table','sort')
                cursor.startRecording(10)
                d3.select('#undo').on 'click', ()-> cursor.undo(); createtable()
                #d3.select('#redo').on 'click', ()-> cursor.redo()
                cell = d3.select(@).selectAll('a').data [1]
                cell.enter().append 'a'
                cell.text d
                    .on 'click', ()=> 
                        debugger
                        tree.set ['table','sort','ascending'], (2+tree.get(['table','sort','ascending']))%3-1
                        tree.set ['table','sort','columns'], [d]
                        createtable()

        index = d3.range tree.get('table','start'), tree.get('table','start') + tree.get('table','rows')
        index = index.map (i)->tree.get('table','index',i)
        rows = table.selectAll('tr.rows').data index
        rows.enter().append('tr').classed 'rows',yes
        rows.each (i)->
            cols = d3.select(@).selectAll('td').data tree.get('table','columns').map (k)=> tree.get 'ColumnDataSource',k,'data',i
            cols.enter().append 'td'
            cols.text (d)-> d

    createtable()
""")))

<IPython.core.display.Javascript object>

# Create a table using d3 and baobab

In [ ]:
create_table = IPython.display.Javascript(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->

    table = d3.select '#viewport'
        .selectAll 'table'
        .html ''
        
        .data [1]
        .enter().append('table')
        
    tree.set 'table', 
        maxRows: 20 
        indexStart: 0
        indexCurrent: 0
        data: Baobab.monkey ['data'],['.','indexStart'],['.','maxRows'], (data,start,rows)->
                d3.range parseInt(start),parseInt(start)+rows
                    .map (i)=> data[i]
        index: Baobab.monkey ['index'],['.','indexStart'],['.','maxRows'], (index,start,rows)->
                d3.range parseInt(start),parseInt(start)+rows
                    .map (i)=> index[i]

        
    table.append('tr').classed {'column_names':true}
        .selectAll('th.cols').data(tree.select('columns').get()).enter().append('th')
        .classed {columns:yes}
        .text (d)-> d
    
    table.selectAll 'tr.rows'
        .data d3.range tree.select('table','maxRows').get()
        .enter().append('tr').classed {'rows':true}

    if tree.select('index').get()
        table.selectAll('tr.rows').each (d,i)->
            d3.select(@).selectAll('th.index').data [tree.select('index').get(i)]
                .enter().append('th').classed {'index':true}
                .text (d)->d
            # Append a column for the rows
            if i == 0
                d3.selectAll('tr.column_names').selectAll('td').data [tree.select('index').get(i)]
                    .enter().insert 'td', ':first-child'
                    
    table.selectAll 'tr.rows'
        .each (d,i)->
            d3.select(@).selectAll('td').data tree.select('data').get(i)
                .enter().append('td').text (d)->d

    
"""))

In [ ]:
IPython.display.display(IPython.display.HTML("""
<div class="row">
<div class="col col-md-6"><h2>Columns</h2><div class="dropper" id="columns" style="height:20px;width:100%;background-color:gray;"></div></div>
<div class="col col-md-6"><h2>Axis</h2><div class="dropper"  id="selection" style="height:20px;width:100%;background-color:lightgray;"></div></div>
<div class="col col-md-12">
<div id="viewport" style="background-color:cyan;"></div>
</div>
</div>""" ),create_table)

# Add virtual scrolling and resizable columns

In [ ]:
display(IPython.display.Javascript(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->
    d3.longscroll = ()->
        [rowHeight,render,size] = [20,null,tree.select('data').get().length]
        offset = d3.scale.linear()
        longscroll = (table)->
            rowHeight = d3.max table.selectAll('tr.rows')[0].map (_this)-> _this.offsetHeight

            offset.domain [0,size]
                .range [0,size*rowHeight]

            parent = d3.select table.property 'parentNode'
                .style 'height', "#{table.node().offsetHeight}px"
                .style 'top', "#{table.node().offsetTop}px"

            parent.selectAll('.longscroll').remove()

            before = parent.selectAll('div.before').data([0])
            before.enter().insert('div',':first-child').classed {'before':yes,'longscroll':yes}

            current = parent.selectAll('div.after').data([0])
            current.enter().append('div').classed {'current':yes,'longscroll':yes}

            after = parent.selectAll('div.after').data([0])
            after.enter().append('div').classed({'after':yes,'longscroll':yes})

            parent.on 'scroll.longscroll', ()-> 
                position= Math.floor offset.invert @scrollTop
                scroll @scrollTop

            scroll = (scrollTop)->
                current.each ()->
                    @scrollTop = scrollTop
                    position0 = d3.max [0, d3.min [size - tree.select('table','maxRows').get(), Math.floor offset.invert(scrollTop)]]
                    before.style("height", "#{offset position0}px")
                    after.style("height", "#{offset (size-position0)}px")
                    tree.select('table','indexStart').set position0
                    table.call(render)
            scroll(0)

        `longscroll.render = function(_) {
        return arguments.length ? (render = _, longscroll) : render;
        };
        longscroll.rowHeight = function(_) {
        return arguments.length ? (rowHeight = +_, longscroll) : rowHeight;
        };
        longscroll.position = function(_) {
        return arguments.length ? (position = +_, longscroll) : position;
        };
        longscroll.size = function(_) {
        return arguments.length ? (size = +_, longscroll) : size;
        };`
        longscroll

    render = ()->
        if Math.abs(tree.select('table','indexStart').get() - tree.select('table','indexCurrent').get()) > 0 
            @selectAll 'tr.rows'
                .data d3.range tree.select('table','maxRows').get()
                .each (r)->                 
                    r = parseInt(r)
                    if (tree.select('index').exists())
                        d3.select(@).selectAll('th').data([tree.select('table','index').get(r)]).text (i)-> i
                    d3.select(@).selectAll('td').data( tree.select('table','data').get(r) ).text (i)-> i

    table = d3.selectAll 'table'
        .call d3.longscroll().render render
        .style {'position':'absolute','left':0,'top':0}

    $(table.node()).colResizable {liveDrag:yes,fixed:yes}
    d3.select('.JCLRgrips').style {'position':'absolute','left':0,'top':0}


""")))

## Create derived ColumnDataSource from split data and generate statistics

In [ ]:
IPython.display.Javascript(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->
    cols = tree.select('columns').get()
    tree.set 'ColumnDataSource', {}
    tree.set 'stats', {} 
    cols.forEach (column_name)->
        tree.set ['ColumnDataSource',column_name], 
            name: column_name
            data: Baobab.monkey ['data'],['columns'],['.','name'], (data, columns, name)->
                index = columns.indexOf name
                data.map (d)-> d[index]
        'min,max,sum,mean,median,variance,deviation'.split(',').forEach (method)=>
            tree.set ['stats',column_name,method], Baobab.monkey ['ColumnDataSource',column_name,'data'], (cds)-> d3[method] cds
            
"""))

# Show some derived results

In [ ]:
display(IPython.display.HTML("""<ul id="summary"></ul>"""),
IPython.display.Javascript(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->
    cols = tree.select('columns').get()
    d3.select '#summary'
        .html ''
        .selectAll('li').data d3.keys(tree.select(['stats']).get())
        .enter()
        .append('li').each (col)->
            d3.select @
                .append 'h1'
                .text col
            d3.select(@).append('ul').selectAll('li').data d3.entries tree.select(['stats',col]).get()
                .enter()
                .append('li').html (d)->"<b>#{d.key}</b> - #{d.value}"
""")))

In [ ]:
%%yaml plot -jtrue
options:
    title: "Rects Demo"
    plot_width: 800
    plot_height: 500
sources: {}
glyphs: 
  - type: Circle
    source: tree
    fill_color: blue
    x: 
        field: sepal_length
    y: 
        field: petal_width
    size: 10
guides: 
  - type: auto
    location: left
    grid: yes
  - type: auto
    location: below
    grid: no
tools: ["Pan", "WheelZoom" ,"Resize" ,"PreviewSave"]

In [ ]:
display(IPython.display.HTML("""<div id="bokeh"></div>"""),
IPython.display.Javascript(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->
    window._yaml = 
        plot: %s
    _yaml.plot.sources.tree = {}
    tree.select('columns').get().forEach (c)=>
            _yaml.plot.sources.tree[c] = tree.select(['ColumnDataSource',c,'data']).get()
    window.bokeh = Bokeh.$('#bokeh').bokeh 'figure', _yaml.plot
    console.log tree.select(['stats',_yaml.plot.glyphs[0].x.field,'min']).get()
    bokeh.attributes.x_range.set 'start', tree.select(['stats',_yaml.plot.glyphs[0].x.field,'min']).get()
    bokeh.attributes.x_range.set 'end', tree.select(['stats',_yaml.plot.glyphs[0].x.field,'max']).get()

    bokeh.attributes.y_range.set 'start', tree.select(['stats',_yaml.plot.glyphs[0].y.field,'min']).get()
    bokeh.attributes.y_range.set 'end', tree.select(['stats',_yaml.plot.glyphs[0].y.field,'max']).get()

"""%json.dumps(plot))))

In [ ]:
(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->
    index = tree.select('index')
    index.set( 
        d3.zip index.get(), tree.select('data').map (c)-> c.get 2
            .sort (a,b)-> a[1]-b[1]
            .map (d)-> d[0]
    )
"""))

In [ ]:
display(
IPython.display.HTML("""<div id="test"></div> """),
IPython.display.Javascript(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->
  N= 630
  N= 50
  r = new Bokeh.Random(123456789)

  xs = ( (x/50) for x in _.range(N) )
  ys = (Math.sin(x) for x in xs)
  color = ("rgb(#{ Math.floor(155+100*val) }, #{ Math.floor(100+50*val) }, #{ Math.floor(150-50*val) })" for val in ys)
  xs = (r.randf()*2 for i in _.range(N))
  ys = (r.randf()*2 for i in _.range(N))
  console.log color
  data = {
    x: xs
    y: ys
    color: color
  }

  rects = {
    type: 'Rect'
    source: "data"
    x: {field: 'x'}
    y: {field: 'y'}
    width:  0.05
    height: 0.05
    angle: 0.1
    line_color: 'black'
    line_width: 3
    fill_color: 'white'
  }

  xaxis = {
    type: "auto"
    location: "below"
    grid: false
  }

  yaxis = {
    type: "auto"
    location: "left"
    grid: true
  }

  options = {
    title: "Rects Demo"
    plot_width: 800
    plot_height: 500
  }

  Bokeh.$("#test").bokeh("figure", {
    options: options
    sources: { data: data }
    glyphs: [rects]
    guides: [xaxis, yaxis]
    tools: ["Pan", "WheelZoom" ,"Resize" ,"PreviewSave"]
  })

""")))